In [14]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
import cv2
from PyQt5 import QtCore
from PyQt5 import QtWidgets
import sip

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# in case you are in debug mode
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

# use this function to create a Viewer, solves the problem of window not raising
viewer=None
def CreateViewer():
    global viewer
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    return viewer

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets', 'PyQt5.QtSvg']


In [15]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
import time
time.sleep(2)

Define some utilities to perform viewer animations

In [16]:
# //////////////////////////////////////////////////////////////
class RunActions:
    
    def __init__(self):
        self.v=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
        
    def addAction(self, fn,**kwargs):
        self.v.append(lambda : fn(**kwargs))
        
    def addSleep(self,msec):
        self.v.append(msec)
        
    def onTimer(self):
        
        # if the last action was an action and the viewer is still running, I need to wait
        if self.cursor>0 and callable(self.v[self.cursor-1]) and viewer.isRunning():
            return
        
        self.timer.stop()
        
        if self.cursor>= len(self.v):
            return
        
        cur=self.v[self.cursor]
        self.cursor+=1
        
        # is a delay?
        if isinstance(cur,int):
            print("*** Sleep",cur)
            self.timer.start(cur) 
            
        # is a real action, run it and later wait for completition
        elif callable(cur):
            cur()
            self.timer.start(20) 
            
        # there must be a problem
        else:
            raise Exception("internal error")
   
    def start(self): 
        self.timer.start(1) 
        
    def stop(self):
        self.timer.stop()
        
print("Utilities defined")

Utilities defined


Some predefined actions

In [17]:
def RotateScene(axis=(0,0,1),angle=10):
    print("*** RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    
def TakeSnapshot(filename="temp.png",):
    print("*** TakeSnapshotAction",filename)  
    # viewer.takeSnapshot(False,filename) BROKEN
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
    screenshot.save(filename)
    
def OpenScene(filename=""):
    print("*** OpenScene",filename)
    viewer.open(filename)
    
def DropSelection():
    print("*** DropSelection")
    viewer.dropSelection()
    
def HideDatasetBounds(uuid="dataset"):
    print("*** HideDatasetBounds",uuid)
    # **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
    dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID(uuid))
    if dataset_node:
        dataset_node.setShowBounds(False)
    else:
        print("Failed to find a node with uuid",uuid)

In [18]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [19]:
filename=r"D:\visus_demo\scenes\battery.xml"
actions=RunActions()
actions.addAction(OpenScene,filename=filename)
actions.addAction(DropSelection)
actions.addAction(HideDatasetBounds,uuid="dataset")
for I in range(5):
    actions.addAction(RotateScene,axis=(0,0,1),angle=10)
    actions.addAction(TakeSnapshot,filename=os.path.splitext(filename)[0] + ".{:03d}.png".format(I))
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle    
    actions.addSleep(50) 
    
actions.start()

*** OpenScene D:\visus_demo\scenes\battery.xml
2022-04-30 12:37:05.284732 PyScriptingNode Got in input (160, 131, 270) float32 origin IsoContour 
2022-04-30 12:37:05.287228 PyScriptingNode Output is  (160, 131, 270) float32 msec 3 
2022-04-30 12:37:05.287728 PyScriptingNode Got in input (160, 16, 143) float32 origin Volume 
2022-04-30 12:37:05.290229 PyScriptingNode Got in input (1, 246, 368) float32 origin Slice 
2022-04-30 12:37:05.292229 PyScriptingNode Output is  (1, 246, 368) float32 msec 2 
2022-04-30 12:37:05.297730 PyScriptingNode Output is  (160, 16, 143) float32 msec 10 
2022-04-30 12:37:05.450759 PyScriptingNode Got in input (320, 262, 540) float32 origin IsoContour 
2022-04-30 12:37:05.453274 PyScriptingNode Got in input (1, 246, 368) float32 origin Slice 2022-04-30 12:37:05.454757 PyScriptingNode Output is  (320, 262, 540) float32 msec 5 

2022-04-30 12:37:05.455257 PyScriptingNode Output is  (1, 246, 368) float32 msec 2 
2022-04-30 12:37:05.455757 PyScriptingNode Got in i

2022-04-30 12:37:21.099961 PyScriptingNode Got in input (640, 124, 569) float32 origin Volume 
2022-04-30 12:37:21.311498 PyScriptingNode Got in input (1, 339, 1024) float32 origin Slice 
2022-04-30 12:37:21.313998 PyScriptingNode Output is  (1, 339, 1024) float32 msec 3 
2022-04-30 12:37:21.434519 PyScriptingNode Got in input (1, 339, 1061) float32 origin Slice 
2022-04-30 12:37:21.436519 PyScriptingNode Output is  (1, 339, 1061) float32 msec 2 
2022-04-30 12:37:21.474526 PyScriptingNode Got in input (640, 525, 1080) float32 origin IsoContour 
2022-04-30 12:37:21.477540 PyScriptingNode Output is  (640, 525, 1080) float32 msec 3 
2022-04-30 12:37:21.938109 PyScriptingNode Output is  (640, 124, 569) float32 msec 838 
*** TakeSnapshotAction D:\visus_demo\scenes\battery.003.png
*** Sleep 50
*** RotateScene (0, 0, 1) 10
*** TakeSnapshotAction D:\visus_demo\scenes\battery.004.png
2022-04-30 12:37:25.741029 PyScriptingNode Got in input (40, 33, 68) float32 origin IsoContour 
2022-04-30 12:37

2022-04-30 13:31:40.401338 PyScriptingNode Got in input (1, 64, 64) float32 origin Slice 
2022-04-30 13:31:40.402837 PyScriptingNode Output is  (1, 64, 64) float32 msec 2 
2022-04-30 13:31:40.490853 PyScriptingNode Got in input (1, 64, 64) float32 origin Slice 
2022-04-30 13:31:40.492855 PyScriptingNode Output is  (1, 64, 64) float32 msec 2 
2022-04-30 13:31:40.551364 PyScriptingNode Got in input (1, 64, 64) float32 origin Slice 
2022-04-30 13:31:40.553364 PyScriptingNode Output is  (1, 64, 64) float32 msec 2 
2022-04-30 13:31:40.612875 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:40.615376 PyScriptingNode Output is  (1, 128, 128) float32 msec 2 
2022-04-30 13:31:40.677386 PyScriptingNode Got in input (1, 64, 64) float32 origin Slice 
2022-04-30 13:31:40.678886 PyScriptingNode Output is  (1, 64, 64) float32 msec 2 
2022-04-30 13:31:40.740397 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:40.741897 PyScriptingNode O

2022-04-30 13:31:50.427664 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:50.429165 PyScriptingNode Output is  (1, 128, 128) float32 msec 2 
2022-04-30 13:31:50.490175 PyScriptingNode Got in input (1, 256, 256) float32 origin Slice 
2022-04-30 13:31:50.491676 PyScriptingNode Output is  (1, 256, 256) float32 msec 1 
2022-04-30 13:31:50.598194 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:50.600194 PyScriptingNode Output is  (1, 128, 128) float32 msec 2 
2022-04-30 13:31:50.678708 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:50.680708 PyScriptingNode Output is  (1, 128, 128) float32 msec 2 
2022-04-30 13:31:50.749220 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:50.751221 PyScriptingNode Output is  (1, 128, 128) float32 msec 2 
2022-04-30 13:31:50.810232 PyScriptingNode Got in input (1, 128, 128) float32 origin Slice 
2022-04-30 13:31:50.811731 P

2022-04-30 13:38:26.398305 PyScriptingNode Got in input (1, 512, 512) float32 origin Slice 
2022-04-30 13:38:26.401305 PyScriptingNode Output is  (1, 512, 512) float32 msec 3 
2022-04-30 13:38:26.460815 PyScriptingNode Got in input (640, 124, 569) float32 origin Volume 
2022-04-30 13:38:26.737364 PyScriptingNode Got in input (1, 604, 904) float32 origin Slice 
2022-04-30 13:38:26.739864 PyScriptingNode Output is  (1, 604, 904) float32 msec 2 
2022-04-30 13:38:26.823879 PyScriptingNode Got in input (640, 525, 1080) float32 origin IsoContour 
2022-04-30 13:38:26.826379 PyScriptingNode Output is  (640, 525, 1080) float32 msec 2 
2022-04-30 13:38:27.302963 PyScriptingNode Got in input (1, 604, 904) float32 origin Slice 
2022-04-30 13:38:27.304963 PyScriptingNode Output is  (1, 604, 904) float32 msec 2 
2022-04-30 13:38:27.330967 PyScriptingNode Output is  (640, 124, 569) float32 msec 871 
2022-04-30 13:38:53.481620 PyScriptingNode Got in input (160, 135, 270) float32 origin IsoContour 
202

2022-04-30 13:43:44.486569 PyScriptingNode Got in input (1, 512, 512) float32 origin Slice 2022-04-30 13:43:44.489070 PyScriptingNode Got in input (320, 262, 540) float32 origin IsoContour 
2022-04-30 13:43:44.490570 PyScriptingNode Output is  (320, 262, 540) float32 msec 1 

2022-04-30 13:43:44.491070 PyScriptingNode Output is  (1, 512, 512) float32 msec 4 
2022-04-30 13:43:44.491570 PyScriptingNode Got in input (640, 124, 569) float32 origin Volume 
2022-04-30 13:43:44.692605 PyScriptingNode Got in input (1, 604, 904) float32 origin Slice 
2022-04-30 13:43:44.694605 PyScriptingNode Output is  (1, 604, 904) float32 msec 3 
2022-04-30 13:43:44.832630 PyScriptingNode Got in input (640, 525, 1080) float32 origin IsoContour 
2022-04-30 13:43:44.835129 PyScriptingNode Output is  (640, 525, 1080) float32 msec 3 
2022-04-30 13:43:45.006160 PyScriptingNode Got in input (1, 604, 904) float32 origin Slice 
2022-04-30 13:43:45.008160 PyScriptingNode Output is  (1, 604, 904) float32 msec 2 
2022-

In [7]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [ ]:
snapshots=sorted(glob.glob(r"D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot*xml"))

# limit for debugging...
snapshots=snapshots[:2] 

actions=RunActions()
for I,snapshot in enumerate(snapshots):
    print("\t",I,snapshot)
    actions.addAction(OpenScene,filename=snapshot)
    actions.addAction(DropSelection)
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(TakeSnapshot,filename=os.path.splitext(snapshot)[0] + ".png")
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle
    actions.addSleep(10) 
    
actions.start()